In [1]:
pip install speechbrain

Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install --upgrade speechbrain


Note: you may need to restart the kernel to use updated packages.


In [3]:
!git clone https://huggingface.co/speechbrain/spkrec-ecapa-voxceleb

Cloning into 'spkrec-ecapa-voxceleb'...
Filtering content: 100% (2/2)
Filtering content: 100% (2/2), 84.73 MiB | 3.12 MiB/s, done.
Encountered 1 file that should have been a pointer, but wasn't:
	mean_var_norm_emb.ckpt


In [1]:

from speechbrain.pretrained import SpeakerRecognition

model = SpeakerRecognition.from_hparams(
    source="spkrec-ecapa-voxceleb",  # Replace with your actual path
    savedir="speechbrain_model"  # Local directory where SpeechBrain saves files
)


INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
C:\Users\kuksh\anaconda3\lib\inspect.py:746: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
C:\Users\kuksh\AppData\Local\Temp\ipykernel_20600\3075496948.py:1: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import SpeakerRecognition
C:\Users\kuksh\anaconda3\lib\site-packages\speechbrain\utils\fetching.

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder


In [4]:
!pip install torch

In [13]:
import sounddevice as sd
import scipy.io.wavfile as wav 

In [1]:
import torchaudio
import torch
import speechbrain as sb
from speechbrain.pretrained import EncoderClassifier
import sounddevice as sd
import numpy as np

# Load the Speaker Recognition Model
model = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    savedir="tmp/spkrec-ecapa-voxceleb"
)

# Function to Record Audio
def record_audio(filename, duration=5, sample_rate=16000):
    print(f"🎤 Recording {duration} seconds of audio...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype=np.float32)
    sd.wait()
    torchaudio.save(filename, torch.tensor(audio_data.T), sample_rate)
    print(f"✅ Recording saved as {filename}")

# Function to Denoise Audio (Placeholder)
def denoise_audio(waveform, sample_rate):
    return waveform  # Implement noise reduction here if needed

def enroll_speaker(audio_paths):
    embeddings = []
    for path in audio_paths:
        waveform, sample_rate = torchaudio.load(path)
        waveform = denoise_audio(waveform, sample_rate)  # Apply denoising

        # Ensure waveform has sufficient length
        if waveform.shape[-1] < 3:
            waveform = torch.nn.functional.pad(waveform, (0, 3 - waveform.shape[-1]))

        embedding = model.encode_batch(waveform, wav_lens=None)
        embeddings.append(embedding)

    return torch.stack(embeddings).mean(dim=0)

# Generate 10 Enrollment Audio Filenames
enrolled_audio_paths = [f"enroll_{i}.wav" for i in range(1, 11)]

# Duplicate the 10 Samples to Make 50
duplicated_audio_paths = enrolled_audio_paths * 5  # 10 samples × 5 = 50

# Record 10 Enrollment Samples
for path in enrolled_audio_paths:
    record_audio(path, duration=5)

# Generate Embedding for Enrollment (Now Using 50 Samples)
enrolled_voice = enroll_speaker(duplicated_audio_paths)


INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
C:\Users\kuksh\anaconda3\lib\inspect.py:746: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
C:\Users\kuksh\AppData\Local\Temp\ipykernel_28340\2454809806.py:4: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import EncoderClassifier
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from H

🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_1.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_2.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_3.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_4.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_5.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_6.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_7.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_8.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_9.wav
🎤 Recording 5 seconds of audio...
✅ Recording saved as enroll_10.wav


In [5]:
pip install noisereduce


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import librosa
import threading
import time
import geocoder
from geopy.geocoders import Nominatim  # Import Nominatim properly
import requests
from pydub import AudioSegment
from pydub.playback import play
from twilio.rest import Client
from transformers import Wav2Vec2Processor, Wav2Vec2ForSequenceClassification
import sounddevice as sd
import numpy as np
import wave
import noisereduce as nr 
# Install with: pip install noisereduce

def get_coordinates(place_name):
    geolocator = Nominatim(user_agent="your_app_name")  # Set a user-agent
    location = geolocator.geocode(place_name)
    
    if location:
        return (location.latitude, location.longitude)
    else:
        return None
    


# ------------------- CONFIG -------------------
# Twilio Credentials
ACCOUNT_SID = "AC37965b7adf6eea30841c41fe571d0cb2"
AUTH_TOKEN = "2b97c6cc1df545c033f4388f3e3c524e"
TWILIO_PHONE_NUMBER = "+12185155312"
TO_PHONE_NUMBER = "+918971754654"

# GraphHopper API Key
GRAPHHOPPER_API_KEY = "0dda774a-d9af-4c81-84f0-b1b866609dbc"

# Security Code (User Input)
CORRECT_CODE = input("Set your security code: ")

# Voice Model
model_name = "facebook/wav2vec2-base"  
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)

# Audio File (Alert Sound)
ALERT_SOUND = "alarm.wav"  # Replace with your alert sound
alert_sound = AudioSegment.from_wav(ALERT_SOUND)

# Tracking
# User input
place = input("Enter a location: ")
coordinates = get_coordinates(place)

if coordinates:
    print(f"Coordinates of {place}: {coordinates}")
else:
    print("Location not found.")

playing = False  # Control audio playback

# ------------------- FUNCTIONS -------------------
def play_audio():
    """Play alert sound."""
    global playing
    playing = True
    while playing:
        play(alert_sound)

def get_current_location():
    """Get user's approximate GPS coordinates."""
    g = geocoder.ip('me')
    return g.latlng if g.latlng else (None, None)

def check_path_exists(start_coords, end_coords):
    """Check if a path exists using GraphHopper API."""
    url = f"https://graphhopper.com/api/1/route?point={start_coords[0]},{start_coords[1]}&point={end_coords[0]},{end_coords[1]}&vehicle=car&key={GRAPHHOPPER_API_KEY}"
    response = requests.get(url).json()
    return "paths" in response and len(response["paths"]) > 0

def denoise_audio(waveform, sample_rate):
    """Apply noise reduction to the audio waveform."""
    waveform = waveform.flatten().numpy()  # Convert tensor to NumPy array
    reduced_noise = nr.reduce_noise(y=waveform, sr=sample_rate)
    return torch.tensor(reduced_noise).unsqueeze(0)  # Convert back to tensor

def record_voice(file_name="recorded_voice.wav", duration=3, sr=16000):
    """Record user's voice, denoise it, and save the file."""
    print("🎤 Recording voice...")
    audio = sd.rec(int(duration * sr), samplerate=sr, channels=1, dtype=np.float32)
    sd.wait()

    # Apply denoising
    audio = denoise_audio(torch.tensor(audio.T), sr)

    wave_file = wave.open(file_name, "wb")
    wave_file.setnchannels(1)
    wave_file.setsampwidth(2)
    wave_file.setframerate(sr)
    wave_file.writeframes(audio.numpy().astype(np.int16).tobytes())
    wave_file.close()

    print("✅ Denoised voice recorded.")
    return file_name

def verify_voice(file_path):
    """Verify the recorded voice against a trained model."""
    y, sr = librosa.load(file_path, sr=16000)
    inputs = processor(y, sampling_rate=16000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values).logits

    return torch.argmax(logits, dim=-1).item() == 1  # Assuming class 1 is the user

def send_alert():
    """Send SMS alert via Twilio and start audio recording."""
    client = Client(ACCOUNT_SID, AUTH_TOKEN)
    try:
        message = client.messages.create(
            body="🚨 DANGER! Verification failed. Immediate action required.",
            from_=TWILIO_PHONE_NUMBER,
            to=TO_PHONE_NUMBER
        )
        print(f"🚨 SMS Alert Sent! SID: {message.sid}")
    except Exception as e:
        print(f"⚠️ Error sending SMS: {e}")
    
    # Start recording evidence audio after alert
    print("🎤 Recording emergency audio...")
    record_voice("emergency_recording.wav", duration=10)  # 10-second recording
    print("✅ Emergency recording saved.")

def verification():
    """Perform verification using code and voice."""
    global playing

    for attempt in range(3):
        # Play alert sound in a separate thread
        audio_thread = threading.Thread(target=play_audio, daemon=True)
        audio_thread.start()

        # Ask for security code
        code = input("\nEnter security code: ")
        if code != CORRECT_CODE:
            print("❌ Incorrect code!")
            continue

        # Record and verify voice
        voice_file = record_voice()
        if not verify_voice(voice_file):
            print("❌ Voice mismatch!")
            continue

        # If both are correct, stop alarm
        print("✅ Verification successful. Stopping alert.")
        playing = False
        return

    # If all 3 attempts fail, send alert
    print("❌ Verification failed 3 times! Sending alert...")
    send_alert()

def track_location():
    """Track user's location, check for path, and trigger verification if needed."""
    last_location = get_current_location()
    if last_location == (None, None):
        print("❌ Could not get current location. Exiting...")
        return

    print(f"Tracking started at location: {last_location}")
    start_time = time.time()
    verification_timer = start_time  # Timer for periodic verification

    while True:
        time.sleep(1)  # Check location every second
        current_location = get_current_location()

        # Check if path exists
        if not check_path_exists(current_location, coordinates):
            print("❌ Path Not Found!")
            continue  # Skip verification if path does not exist

        # Check if the user has been stationary for 10 seconds
        if current_location == last_location:
            elapsed_time = time.time() - start_time
            if elapsed_time >= 10:
                print("🚨 Person stationary for 10 seconds. Starting verification!")
                verification()
                start_time = time.time()  # Reset timer

        else:
            last_location = current_location
            start_time = time.time()  # Reset stationary timer

        # Trigger verification every 20 seconds
        if time.time() - verification_timer >= 20:
            print("⏳ Time-based verification started...")
            verification()
            verification_timer = time.time()  # Reset verification timer

        # If user is unresponsive for 1 minute, send an alert
        if time.time() - start_time >= 60:
            print("⏳ User unresponsive for 1 minute! Sending alert...")
            send_alert()
            break

# ------------------- START TRACKING -------------------
track_location()


C:\Users\kuksh\anaconda3\lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\kuksh\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\kuksh\anaconda3\lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,
C:\Users\kuksh\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't

Set your security code: 1212


C:\Users\kuksh\anaconda3\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\kuksh\anaconda3\lib\site-packages\transformers\configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
C:\Users\kuksh\anaconda3\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing 

Enter a location: bangalore
Coordinates of bangalore: (12.98815675, 77.62260003796)


INFO:geocoder.base:Requested http://ipinfo.io/json


...

⏳ Time-based verification started...

Enter security code: 1212
🎤 Recording voice...
✅ Denoised voice recorded.
✅ Verification successful. Stopping alert.


INFO:geocoder.base:Requested http://ipinfo.io/json


🚨 Person stationary for 10 seconds. Starting verification!

Enter security code: 1212
🎤 Recording voice...
✅ Denoised voice recorded.
✅ Verification successful. Stopping alert.


INFO:geocoder.base:Requested http://ipinfo.io/json


KeyboardInterrupt: 